In [47]:
pip install bigml

Note: you may need to restart the kernel to use updated packages.


In [48]:
import bigml.api

In [49]:
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [50]:
source = api.create_source('fulltrain_dataframe.csv')

In [51]:
api.ok(source)

True

In [52]:
dataset_1 = api.create_dataset(source)

In [53]:
api.ok(dataset_1)

True

In [54]:
train_dataset = api.create_dataset(
    dataset_1, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_1, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [55]:
api.ok(train_dataset)

True

In [56]:
model = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [57]:
api.ok(model)

True

In [58]:
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True})

In [59]:
api.ok(batch_prediction)

True

In [60]:
api.download_batch_prediction(batch_prediction,
    filename='../handson-ml2/prediction_credit.csv')

'../handson-ml2/prediction_credit.csv'

In [61]:
test_source = api.create_source('test_dataframe.csv')

In [62]:
api.ok(test_source)

True

In [63]:
dataset_2 = api.create_dataset(test_source)

In [64]:
api.ok(dataset_2)

True

In [65]:
model = api.create_ensemble(dataset_1, {"objective_field":"SeriousDlqin2yrs"})

In [66]:
api.ok(model)

True

In [67]:
batch_prediction = api.create_batch_prediction(model, dataset_2, {"all_fields": True, "confidence": True})

In [68]:
api.ok(batch_prediction)

True

In [69]:
evaluation = api.create_evaluation(model, dataset_1)


In [70]:
api.ok(evaluation)

True

In [71]:
api.download_batch_prediction(batch_prediction,
    filename='../Ex_credit_python/prediction_credit_final.csv')

'../Ex_credit_python/prediction_credit_final.csv'